# Retrieving NTD Ridership Data

This notebook explores how to download ridership data from the National Transit Database. Data Source from the [American Public Transportation Association.](https://www.apta.com/research-technical-resources/transit-statistics/ntd-data-tables/)

In [5]:
# https://www.apta.com/wp-content/uploads/2020-National-Transit-Database.zip

import os
import pandas as pd
from io import BytesIO
from zipfile import ZipFile
import requests

In [84]:
# Set up some variables for the years of interest and the output directory
years = [2013] + list(range(2015, 2021))
output_dir = '../data/raw/ntd-ridership'

Note some differences in filepaths:
- 2013: `2013-National-Transit-Database/2013-Table-19-Transit-Operating-Stats.xls`
- ~~2014: `2014-National-Transit-Database/2014-Table-19-Transit-Operating-Stats.xls`~~ *Error with macros, download manually*
- 2015: `2015-NTD-Tables-APTA/Metrics.xlsm`
- 2016: `2016-National-Transit-Database/2016-NTD-Metrics_0.xlsx`
- 2017: `2017-National-Transit-Database/Metrics_1.xlsm`
- 2018: `2018-National-Transit-Database/Metrics_2.xlsx`
- 2019: `2019_Annual_Database_Files/Metrics_Static.xlsx`
- 2020: `2020_Annual_Database_Files/Metrics_Static.xlsx`

In [58]:
file_paths = {
    2013: '2013-Table-19-Transit-Operating-Stats.xls',
    2014: '2014-Table-19-Transit-Operating-Stats.xls',
    2015: 'Metrics.xlsm',
    2016: '2016-NTD-Metrics_0.xlsx',
    2017: 'Metrics_1.xlsm',
    2018: 'Metrics_2.xlsx',
    2019: '2019_Annual_Database_Files/Metrics_Static.xlsx',
    2020: '2020_Annual_Database_Files/Metrics_Static.xlsx'
}

In [81]:
def get_zip(year):
    if year == 2017:
        url = requests.get('https://www.apta.com/wp-content/uploads/Resources/resources/statistics/Documents/NTD_Data/2017-National-Transit-Database.zip')
    elif year == 2015:
        url = requests.get('https://www.apta.com/wp-content/uploads/2015-NTD-Tables-APTA.zip')
    else:
        url = requests.get(f'https://www.apta.com/wp-content/uploads/{year}-National-Transit-Database.zip')

    zipfile = ZipFile(BytesIO(url.content))

    with zipfile.open(file_paths[year]) as f:
        if year == 2013:
            df = pd.read_excel(f, sheet_name="Op_Stats_Service", skiprows=[0])
        # elif year == 2014:
        #     df = pd.read_excel(f, sheet_name="Op_Stats_Service", skiprows=[0, 3])
        else:
            df = pd.read_excel(f, sheet_name="Metrics")

    # Filter the DataFrame to rows where column J == 'RB'
    df_filtered = df[df["Mode"] == "RB"]

    # Save the filtered data to a new CSV file
    output_file = os.path.join(output_dir, f'transit_data_{year}_filtered.csv')
    df_filtered.to_csv(output_file, index=False)

In [85]:
for year in years:
    print(year)
    get_zip(year)

2013
2015
2016
2017
2018
2019
2020


In [50]:
# url = requests.get(f'https://www.apta.com/wp-content/uploads/2013-National-Transit-Database.zip')
# zipfile = ZipFile(BytesIO(url.content))

**Manual cleaning required for 2013 and 2014:** 
- In the **2014** CSV, `Column O` onward is split because of prior cells being merged. Take the second row for these cols and override the first
- Take these same headings and override the first row of the **2013** CSV from `Column M` onward